# Initialize

# Initialization

In [ ]:
pip install langchain openai tiktoken slack_sdk python-slugify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["SLACK_BOT_TOKEN"] = ""
os.environ["SLACK_BOT_CHANNEL"] = ""
os.environ["DEBUG_MODE"] = "TRUE"
os.environ["LLM_TEMP"] = "0"
os.environ["LLM_CREATIVE_TEMP"] = "1"

In [ ]:
import os
from langchain.llms import OpenAI

llm = OpenAI(temperature=int(os.environ["LLM_TEMP"]))
llm_creative = OpenAI(temperature=int(os.environ["LLM_CREATIVE_TEMP"]))

In [ ]:
import os
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

slack_token = os.environ["SLACK_BOT_TOKEN"]
slack_client = WebClient(token=slack_token)

# Slack

In [ ]:
def sendMessageToChannel (text="", blocks=None, previous_response=None):
  thread_ts = None if previous_response is None else previous_response['ts']

  return slack_client.chat_postMessage(
        channel=os.environ["SLACK_BOT_CHANNEL"],
        text=text,
        blocks=blocks,
        type="mrkdwn",
        thread_ts=thread_ts
  )

In [ ]:
def getSlackMessageAsBlocks (title, cta, documents_summarized_list):
  blocks = [
    {
			"type": "header",
			"text": {
				"type": "plain_text",
				"text": title,
				"emoji": True
			}
		},
  ]
  
  for index, text in enumerate(documents_summarized_list):
    url = getLinkToDocumentByFileIndex(index)
    section_title = getTitleOfDocumentByFileIndex(index)

    blocks.append({
			"type": "section",
			"text": {
				"type": "mrkdwn",
				"text": f"{text}"
			},
			"accessory": {
				"type": "button",
				"text": {
					"type": "plain_text",
					"emoji": True,
					"text": "Learn More"
				},
				"url": url,
				"action_id": f"button_{index}"
			}
		})

  blocks.append({
     "type": "divider"
  })

  blocks.append(
		{
			"type": "section",
			"text": {
				"type": "mrkdwn",
				"text": cta
			}
  })

  return blocks

In [ ]:
def getSlackMessageAsPlainText (title, cta, documents_summarized_short):
  documents_summarized_list = [d.replace("\n", "- ") for d in documents_summarized_short]
  documents_summarized_list_with_links = [f"{d} `<{getLinkToDocumentByFileIndex(i)}|*[link]*>`" for i, d in enumerate(documents_summarized_list)]
  documents_summarized_combined = "\n".join(documents_summarized_list_with_links)

  message_template = Template('*$title*\n\n$documents_summarized_combined\n\n$cta')
  return message_template.safe_substitute(title=title, documents_summarized_combined=documents_summarized_combined, cta=cta)


# Texts utils

In [ ]:
from datetime import date, timedelta

def getFormattedCurrentDate ():
    today = date.today() + timedelta(days=-1)
    return today.strftime("%d-%m-%Y")

In [ ]:
from glob import glob
from string import Template
from langchain.docstore.document import Document

def getFilesFromPreviousDay ():
  currentDate = getFormattedCurrentDate()
  path_template = Template('drive/MyDrive/prasowka/$currentDate/*')
  path = path_template.substitute(currentDate=currentDate)
  return glob(path)

def getFileNameFromPreviousDayByIndex (index):
  files = getFilesFromPreviousDay()
  return files[index].split("/").pop()

def getDocumentsTexts ():
  files = getFilesFromPreviousDay()

  for file_key, file in enumerate(files):
    with open(file) as f: contents = f.read()
    files[file_key] = contents

  return files

def mapTextsToDocuments (texts):
  return [Document(page_content=t) for t in texts]

def getDocuments ():
  texts = getDocumentsTexts()
  return mapTextsToDocuments(texts)

In [ ]:
from slugify import slugify

def getLinkToDocumentByFileIndex (index):
  file_name = getFileNameFromPreviousDayByIndex(index).replace(".md", "")
  file_name_slug = slugify(file_name)
  return (f"https://www.notion.so/appunite/{file_name_slug}")

def getTitleOfDocumentByFileIndex (index):
  files = getFilesFromPreviousDay()
  with open(files[index]) as f: title = f.readline()
  title_without_formatting = title.replace("# ", "").strip()
  
  return title_without_formatting

# Prompts

In [ ]:
from langchain.prompts import PromptTemplate

SUMMARIZE_SINGLE_LONG_PROMPT_TEMPLATE = """Summarize text below in points. Provide important information.

Summarize Title
Summarize "Context"
Summarize "What problems do we aim to solve"
Summarize "Hypothesis"
Summarize "Implementation plan"
Summarize "Why the owner(s) should make this decision"

{text}
"""

SUMMARIZE_SINGLE_SHORT_PROMPT_TEMPLATE = """
Summarize the following information in a short, catchy, engaging sentence in English. Use one emoji referring to the text but not the decision.

Examples (DO NOT USE THEM):
- 😒 Our health industry client in the USA is proving to be a bit difficult with their lack of openness and strange demands. Plus, they won't even review our NDA template. Can we really trust them?
- 📈 We're migrating to GA4 and creating a Slack channel to keep things organized. Efficiency, baby!
- 🤝 Our improved exit interview process will give us valuable feedback and keep the team running smoothly.
- 🚀 We're launching a new product next month and our team is working tirelessly to make it a success. Buckle up, it's going to be a wild ride!
- 👥 We're expanding our team and hiring for several positions. Know anyone who would be a perfect fit? Let us know!

TEXT:
{text}
"""

SUMMARIZE_TITLE_DAY_PROMPT_TEMPLATE = """
Write similar title of a message which will include a summary of decisions from yesterday. Use emojis.

Examples (do not use them):
🔥 We made some 🔝 moves yesterday! Check out what's new!
🎉 Exciting news! Big things were made yesterday! 🙌
🤯 Hold onto your hats! Yesterday's documents will blow your mind!
🚀 Blast off into the future! Yesterday's decisions will take us to new heights! 🚀
💥 Brace yourselves! We've got some major updates that will blow your mind! 
🎊 Hold onto your seats, exciting news is coming your way! 🎉
😱 Get ready to be amazed! We've got some incredible developments to share with you!
🌟 It's time to shine! Our latest moves will take us to the next level! ✨
🚀 Buckle up, we're launching into a new era of success! 🚀


"""

SUMMARIZE_CTA_DAY_PROMPT_TEMPLATE = """
Write a catchy sentence ending a summary of decisions from yesterday. Use emojis.


"""

SUMMARIZE_SINGLE_LONG_PROMPT = PromptTemplate(template=SUMMARIZE_SINGLE_LONG_PROMPT_TEMPLATE, input_variables=["text"])
SUMMARIZE_SINGLE_SHORT_PROMPT = PromptTemplate(template=SUMMARIZE_SINGLE_SHORT_PROMPT_TEMPLATE, input_variables=["text"])
SUMMARIZE_TITLE_DAY_PROMPT = PromptTemplate(template=SUMMARIZE_TITLE_DAY_PROMPT_TEMPLATE, input_variables=[])
SUMMARIZE_CTA_DAY_PROMPT = PromptTemplate(template=SUMMARIZE_CTA_DAY_PROMPT_TEMPLATE, input_variables=[])

# Summarization of texts

In [ ]:
import asyncio

async def promptSingleText (chain, text):
  return await chain.arun({"input_documents": [text], "text": text})

async def promptMultipleTexts (chain, texts):
  texts_prompt_promise = [promptSingleText(chain, t) for t in texts]
  texts_prompt = await asyncio.gather(*texts_prompt_promise)
  return texts_prompt

# Summarization of whole day

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import LLMChain

# summarize chain provides auto text-splitting but we need to research more how it works
# getSummarizeIndividualLongChain = lambda: load_summarize_chain(llm, chain_type="map_reduce", map_prompt=SUMMARIZE_SINGLE_LONG_PROMPT)
summarize_individual_long_chain = LLMChain(llm=llm, prompt=SUMMARIZE_SINGLE_LONG_PROMPT)
# getSummarizeIndividualShortChain = lambda: load_summarize_chain(llm, chain_type="map_reduce", map_prompt=SUMMARIZE_SINGLE_SHORT_PROMPT)
summarize_individual_short_chain = LLMChain(llm=llm, prompt=SUMMARIZE_SINGLE_SHORT_PROMPT)
#
summarize_title_day_chain = LLMChain(llm=llm_creative, prompt=SUMMARIZE_TITLE_DAY_PROMPT)
#
summarize_cta_day_chain = LLMChain(llm=llm_creative, prompt=SUMMARIZE_CTA_DAY_PROMPT)

async def getSummarizedLong (documents):
  return await promptMultipleTexts(summarize_individual_long_chain, documents)

async def getSummarizedShortOfLong (documents):
  return await promptMultipleTexts(summarize_individual_short_chain, documents)

async def getSummarizeTitle ():
  return await promptSingleText(summarize_title_day_chain, '')

async def getSummarizeCTA ():
  return await promptSingleText(summarize_cta_day_chain, '')

async def getSummarizedAllDayMessage (documents, type="text"):
  day_summarize_title, day_summarize_cta, documents_summarized_long = await asyncio.gather(*[getSummarizeTitle(), getSummarizeCTA(), getSummarizedLong(documents)])

  day_summarize_title_escaped = day_summarize_title.replace('"', '')
  day_summarize_cta_escaped = day_summarize_cta.replace('"', '')

  documents_summarized_long_parsed = mapTextsToDocuments(documents_summarized_long)
  documents_summarized_short = await getSummarizedShortOfLong(documents_summarized_long_parsed)

  if type == "blocks":
    documents_summarized_list = [d.replace("\n", "") for d in documents_summarized_short]
    return getSlackMessageAsBlocks(day_summarize_title_escaped, day_summarize_cta_escaped, documents_summarized_list) 

  documents_summarized_list = [d.replace("\n", "- ") for d in documents_summarized_short]
  return getSlackMessageAsPlainText(day_summarize_title_escaped, day_summarize_cta_escaped, documents_summarized_list)

# Bootstrap

In [ ]:
def getDebugInfo(time_of_exec):
  return (f"""
*time of execution:* {round(time_of_exec, 2)}s
*llm base temp (summary)*: {os.environ["LLM_TEMP"]}
*llm creative temp (title, cta)*: {os.environ["LLM_CREATIVE_TEMP"]}

---

*beginning prompt*: ```{SUMMARIZE_TITLE_DAY_PROMPT_TEMPLATE}```\n\n
*summary long prompt*: ```{SUMMARIZE_SINGLE_SHORT_PROMPT_TEMPLATE}```\n\n
*summary short prompt*: ```{SUMMARIZE_SINGLE_SHORT_PROMPT_TEMPLATE}```\n\n
*ending prompt*: ```{SUMMARIZE_CTA_DAY_PROMPT_TEMPLATE}```""")

In [ ]:
import time

async def bootstrap ():
  documents = getDocuments()
  no_documents = len(documents) == 0
  
  if no_documents:
    return
  
  start_time = time.time()
  today_message_text = await getSummarizedAllDayMessage(documents)
  end_time = time.time()
  message_slack_response = sendMessageToChannel(text=today_message_text)

  if os.environ["DEBUG_MODE"]:
    details = getDebugInfo(end_time - start_time)
    sendMessageToChannel(text=details, blocks=None, previous_response=message_slack_response)

  print(today_message_text)

await bootstrap()

*🤩 Yesterday's Decisions: Amazing Results Ahead! 🤩*

- 🤝 We're looking to build the best possible team for our cooperation with Dembol on BE, and introducing Damian Dębicki as a Backend Developer and Team Leader could make a big impact! `<https://www.notion.so/appunite/lets-start-opnr-with-dembol-on-be-8ed0c1b95faf4c6ca5c72c2548765e95|*[link]*>`
- 🤝 Adding Semir to the ReverlyLabs team will help the current developers, convince the client of the value of a larger team, and ultimately make a difference - all by May 5, 2023! `<https://www.notion.so/appunite/we-would-like-to-add-semir-to-reverlylabs-team-eb109641c502448c8545a9a865464c94|*[link]*>`
- 🔍 We're experimenting with a new tool to give our team context and help prioritize tasks for better weekly planning. 🧪 `<https://www.notion.so/appunite/marketing-will-start-using-different-approach-to-p-f5877ceb29ec48c49ebfda4874c95043|*[link]*>`
- 🤝 Tadeusz Miesiac is proposed to take the role of owner of The Bench team to solve the problem o